# Preprocessing Pipeline


In [1]:
from pathlib import Path
import json

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)


In [2]:

DROP_COLS = [
    'AvgWholesale',
    'AvgRetail',
    'GoodWholesale',
    'GoodRetail',
    'TradeMin',
    'TradeMax',
    'PrivateMax',
]

CORE_NUMERIC_COLS = ['NewPrice', 'Sold_Amount', 'Age_Comp_Months', 'KM']


def load_raw_datasets(train_path, test_path):
    train_df = pd.read_csv(train_path, sep='\t', encoding='utf-8-sig', low_memory=False)
    test_df = pd.read_csv(test_path, sep='\t', encoding='utf-8-sig', low_memory=False)

    train_df['dataset_split'] = 'train'
    test_df['dataset_split'] = 'test'
    return pd.concat([train_df, test_df], ignore_index=True)


def filter_dataset(
    df,
    missing_threshold_pct=5.0,
    max_age_months=246,
    drop_zero_target=True,
    drop_target_gt_newprice=True,
):
    out = df.copy()
    info = {'rows_before': int(len(out))}

    out = out.drop(columns=DROP_COLS, errors='ignore')

    missing_pct = (out.isna().mean() * 100).sort_values(ascending=False)
    keep_cols = missing_pct[missing_pct <= float(missing_threshold_pct)].index.tolist()
    out = out[keep_cols]

    for col in CORE_NUMERIC_COLS:
        if col in out.columns:
            out[col] = pd.to_numeric(out[col], errors='coerce')

    if drop_zero_target and 'Sold_Amount' in out.columns:
        out = out[out['Sold_Amount'] != 0]

    if drop_target_gt_newprice and {'Sold_Amount', 'NewPrice'}.issubset(out.columns):
        out = out[out['Sold_Amount'] <= out['NewPrice']]

    if {'Age_Comp_Months', 'NewPrice'}.issubset(out.columns):
        out = out[(out['Age_Comp_Months'] >= 0) & (out['Age_Comp_Months'] <= max_age_months) & (out['NewPrice'] >= 0)]

    out = out.reset_index(drop=True)
    info['rows_after'] = int(len(out))
    info['cols_after'] = int(out.shape[1])
    info['missing_threshold_pct'] = float(missing_threshold_pct)
    info['max_age_months'] = int(max_age_months)
    return out, info


def load_and_filter(train_path, test_path, missing_threshold_pct=5.0, max_age_months=246):
    df = load_raw_datasets(train_path, test_path)
    return filter_dataset(df=df, missing_threshold_pct=missing_threshold_pct, max_age_months=max_age_months)


In [3]:
CAT_COLS = [
    'MakeCode',
    'FamilyCode',
    'BodyStyleDescription',
    'DriveCode',
    'GearTypeDescription',
    'GearLocationDescription',
    'FuelTypeDescription',
    'InductionDescription',
    'BuildCountryOriginDescription',
]

NUM_COLS = [
    'GearNum',
    'DoorNum',
    'EngineSize',
    'Cylinders',
    'FuelCapacity',
    'NewPrice',
    'WarrantyYears',
    'WarrantyKM',
    'KM',
]

TARGET_COL = 'Sold_Amount'

GEAR_TYPE_MAP = {
    'Sports Automatic Single Clutch': 'Sports Automatic',
    'Sports Automatic Dual Clutch': 'Sports Automatic',
    'Seq. Manual Auto-Single Clutch': 'Manual',
    'Manual Auto-clutch - H Pattern': 'Manual',
}

INDUCTION_MAP = {
    'Turbo Intercooled': 'Turbo',
    'Supercharged Intercooled': 'Supercharged',
    'Twin Turbo Intercooled': 'Turbo',
}

FUEL_MAP = {
    'Petrol - Unleaded ULP': 'Petrol',
    'Petrol - Premium ULP': 'Petrol',
    'Petrol or LPG (Dual)': 'Petrol or LPG',
}


def _fill_mode(df, col):
    if col not in df.columns:
        return df
    mode = df[col].mode(dropna=True)
    if not mode.empty:
        df[col] = df[col].fillna(mode.iloc[0])
    return df


def _fill_median(df, col):
    if col not in df.columns:
        return df
    series = pd.to_numeric(df[col], errors='coerce')
    df[col] = series.fillna(series.median())
    return df


def prepare_model_dataframe(df):
    out = df.copy()

    # fill categoricals with mode
    for col in CAT_COLS:
        out = _fill_mode(out, col)

    # fill numericals with median
    for col in NUM_COLS:
        out = _fill_median(out, col)

    for col in [TARGET_COL, 'NewPrice', 'KM']:
        if col in out.columns:
            out[col] = pd.to_numeric(out[col], errors='coerce')
            out = out[out[col].notna()]

    if 'Compliance_Date' in out.columns:
        out = out[out['Compliance_Date'].notna()]

    if 'Sold_Date' in out.columns:
        out['Sold_Date'] = pd.to_datetime(out['Sold_Date'], errors='coerce')
    if 'Compliance_Date' in out.columns:
        out['Compliance_Date'] = pd.to_datetime(out['Compliance_Date'], format='%m/%Y', errors='coerce')

    if {'Sold_Date', 'Compliance_Date'}.issubset(out.columns):
        out = out[out['Sold_Date'].notna() & out['Compliance_Date'].notna()]
        out['AgeDays'] = (out['Sold_Date'] - out['Compliance_Date']).dt.days
    else:
        out['AgeDays'] = pd.to_numeric(out.get('Age_Comp_Months'), errors='coerce') * 30

    if 'GearTypeDescription' in out.columns:
        out['GearTypeDescription'] = out['GearTypeDescription'].replace(GEAR_TYPE_MAP)
    if 'InductionDescription' in out.columns:
        out['InductionDescription'] = out['InductionDescription'].replace(INDUCTION_MAP)
    if 'FuelTypeDescription' in out.columns:
        out['FuelTypeDescription'] = out['FuelTypeDescription'].replace(FUEL_MAP)

    # Keep all remaining columns after preprocessing (do not narrow to 18 columns).
    # Drop raw date columns after deriving AgeDays.
    out = out.drop(columns=['Sold_Date', 'Compliance_Date'], errors='ignore')

    if 'AgeDays' in out.columns and TARGET_COL in out.columns:
        out = out.dropna(subset=['AgeDays', TARGET_COL]).reset_index(drop=True)
        out['AgeDays'] = out['AgeDays'].astype(float)
    elif TARGET_COL in out.columns:
        out = out.dropna(subset=[TARGET_COL]).reset_index(drop=True)

    return out


def label_encode_columns(df, columns):
    out = df.copy()
    label_maps = {}

    for col in columns:
        if col not in out.columns:
            continue
        values = out[col].astype(str)
        labels = sorted(values.unique().tolist())
        mapping = {label: idx for idx, label in enumerate(labels)}
        out[col] = values.map(mapping).astype(int)
        label_maps[col] = mapping

    return out, label_maps


def to_training_matrix(df, label_map_path=None, log_target=True, log_newprice=True):
    out = df.copy()

    cat_cols = out.select_dtypes(include=['object', 'bool']).columns.tolist()
    out, label_maps = label_encode_columns(out, cat_cols)

    if label_map_path is not None:
        label_map_path.parent.mkdir(parents=True, exist_ok=True)
        with open(label_map_path, 'w', encoding='utf-8') as f:
            json.dump(label_maps, f, indent=2, sort_keys=True)

    if log_target and TARGET_COL in out.columns:
        out[TARGET_COL] = np.log1p(out[TARGET_COL].clip(lower=0))

    if log_newprice and 'NewPrice' in out.columns:
        out['NewPrice'] = np.log1p(pd.to_numeric(out['NewPrice'], errors='coerce').clip(lower=0))

    # Ensure all remaining columns are numeric for model ingestion.
    for col in out.columns:
        out[col] = pd.to_numeric(out[col], errors='coerce')

    out = out.dropna().reset_index(drop=True)
    return out, label_maps


In [4]:
def find_project_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / 'data' / 'raw').exists():
            return p
    raise FileNotFoundError('Could not find project root containing data/raw')

PROJECT_ROOT = find_project_root(Path.cwd())
RAW_DIR = PROJECT_ROOT / 'data' / 'raw'
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_PATH = RAW_DIR / 'DatiumTrain.rpt'
TEST_PATH = RAW_DIR / 'DatiumTest.rpt'
FILTERED_PATH = PROCESSED_DIR / 'dataset_filtered.csv'
MODEL_PATH = PROCESSED_DIR / 'dataset_model.csv'
TRAINING_MATRIX_PATH = PROCESSED_DIR / 'dataset_training_matrix.csv'
PREPROCESSING_DIR = PROJECT_ROOT / 'notebooks' / 'preprocessing'
LABEL_MAP_PATH = PREPROCESSING_DIR / 'label_encoding_map.json'

print('PROJECT_ROOT:', PROJECT_ROOT)
print('TRAIN_PATH:', TRAIN_PATH)
print('TEST_PATH:', TEST_PATH)
print('LABEL_MAP_PATH:', LABEL_MAP_PATH)


PROJECT_ROOT: /home/localhost/datium_model
TRAIN_PATH: /home/localhost/datium_model/data/raw/DatiumTrain.rpt
TEST_PATH: /home/localhost/datium_model/data/raw/DatiumTest.rpt
LABEL_MAP_PATH: /home/localhost/datium_model/notebooks/preprocessing/label_encoding_map.json


## 1) Load filtered dataset (EDA-aligned)


In [5]:
filtered_df, summary = load_and_filter(
    train_path=TRAIN_PATH,
    test_path=TEST_PATH,
    missing_threshold_pct=5.0,
    max_age_months=246,
)

print('Filter summary:', summary)
display(filtered_df.head(3))
display(filtered_df[['NewPrice', 'Sold_Amount', 'Age_Comp_Months', 'KM']].describe())


Filter summary: {'rows_before': 62192, 'rows_after': 61292, 'cols_after': 68, 'missing_threshold_pct': 5.0, 'max_age_months': 246}


,WarrantyKM,KerbWeight,WarrantyYears,EngineConfigurationDescription,EngineNum,Series,FrontTyreSize,RearTyreSize,FuelCapacity,FrontRimDesc,RearRimDesc,Height,Length,Width,VIN,ValvesCylinder,VFactsSegment,VFactsClass,TorqueRPMTo,Torque,CamDescription,WheelBase,PowerRPMTo,Power,Colour,Age_Comp_Months,Compliance_Date,SeatCapacity,MethodOfDeliveryDescription,GoodKM,AverageKM,EngineLocation,GearNum,KM,GearLocationDescription,Sold_Amount,NewPrice,BuildCountryOriginDescription,EngineCycleDescription,IsPPlateApproved,Sold_Date,SaleCategory,Branch,FamilyCode,FuelTypeDescription,Cylinders,FuelDeliveryDescription,EngineTypeDescription,InductionDescription,OptionCategory,YearGroup,Make,LimitedEdition,ImportFlag,CurrentRelease,Description,MonthGroup,SequenceNum,EngineSize,EngineDescription,Model,BodyStyleDescription,DriveCode,DriveDescription,GearTypeDescription,DoorNum,MakeCode,dataset_split
0,100000.0,1690.0,3.0,V60,HBA04 ######,VE,225/60 R16,225/60 R16,73.0,16x7.0,16x7.0,1476.0,4894.0,1899.0,6G1EK52B#8L######,4.0,Large,Passenger,2600.0,330.0,DOHC with VVT,2915.0,6000.0,180.0,White,93.0,02/2008,5.0,Electronic Sequential,140.0,230.0,Front,4.0,227878.0,Floor,2000.0,34790.0,AUSTRALIA,4 Stroke,T,2015-11-03 00:00:00.000,Auction,Perth (WA),COMMODO,Petrol - Unleaded ULP,6,Multi-Point Injection,Piston,Aspirated,PASS,2008,Holden,F,L,F,VE Omega Sedan 4dr. Auto 4sp 3.6i,0,0,3565,3.6,Commodore,Sedan,RWD,Rear Wheel Drive,Automatic,4,HOLD,train
1,NaN,1380.0,NaN,V90,VH-######,VR,205/65 R15,205/65 R15,68.0,15x6.0,15x6.0,1478.0,4903.0,1794.0,6H8VRK35HPL######,2.0,Large,Passenger,3200.0,295.0,Pushrod,2822.0,4800.0,130.0,Red,86.0,08/1993,5.0,Electronic,360.0,600.0,Front,4.0,153091.0,Floor,6800.0,27978.0,AUSTRALIA,4 Stroke,T,2000-10-18 00:00:00.000,Auction,Belmore (NSW),COMMODO,Petrol - Unleaded ULP,6,Multi-Point Injection,Piston,Aspirated,PASS,1993,Holden,F,L,F,VR Executive Wagon 5dr. Auto 4sp 3.8i,7,41,3791,3.8,Commodore,Wagon,RWD,Rear Wheel Drive,Automatic,5,HOLD,train
2,100000.0,1545.0,3.0,In-line,2AZ-#######,ACA33R,225/65 R17,225/65 R17,60.0,17x6.5,17x6.5,1695.0,4625.0,1815.0,JTMBD33V*05######,4.0,Medium,SUV,4000.0,224.0,DOHC with VVT,2660.0,6000.0,125.0,040 - Glacier White (T),16.0,10/2012,5.0,Electronic Sequential,80.0,130.0,Front,5.0,27374.0,Floor,22900.0,31990.0,JAPAN,4 Stroke,T,2014-02-05 00:00:00.000,Dealer Only Auction,Sunshine (VIC),RAV4,Petrol - Unleaded ULP,4,Multi-Point Injection,Piston,Aspirated,SUV,2012,Toyota,F,L,F,ACA33R MY12 CV Wagon 5dr Man 5sp 4x4 2.4i,0,6,2362,2.4,RAV4,Wagon,4XO,4X4 On Demand,Manual,5,TOYO,train


,NewPrice,Sold_Amount,Age_Comp_Months,KM
count,61292.000000,61292.000000,61292.000000,6.128800e+04
mean,37068.608432,16915.340573,46.034784,8.466719e+04
std,15662.320374,10388.574715,31.088816,7.077161e+04
min,7115.000000,1.000000,0.000000,0.000000e+00
25%,29500.000000,10887.250000,26.000000,4.073625e+04
50%,34990.000000,15250.000000,39.000000,6.871000e+04
75%,41790.000000,20900.000000,56.000000,1.162020e+05
max,525000.000000,317000.000000,246.000000,8.883234e+06


In [6]:
filtered_df.to_csv(FILTERED_PATH, index=False)
print('Saved filtered dataset:', FILTERED_PATH)


Saved filtered dataset: /home/localhost/datium_model/data/processed/dataset_filtered.csv


## 2) Build model dataframe


In [7]:
model_df = prepare_model_dataframe(filtered_df)
print('Model dataframe shape:', model_df.shape)
display(model_df.head(3))
display(model_df.isna().sum().sort_values(ascending=False).head(15))


Model dataframe shape: (61292, 67)


,WarrantyKM,KerbWeight,WarrantyYears,EngineConfigurationDescription,EngineNum,Series,FrontTyreSize,RearTyreSize,FuelCapacity,FrontRimDesc,RearRimDesc,Height,Length,Width,VIN,ValvesCylinder,VFactsSegment,VFactsClass,TorqueRPMTo,Torque,CamDescription,WheelBase,PowerRPMTo,Power,Colour,Age_Comp_Months,SeatCapacity,MethodOfDeliveryDescription,GoodKM,AverageKM,EngineLocation,GearNum,KM,GearLocationDescription,Sold_Amount,NewPrice,BuildCountryOriginDescription,EngineCycleDescription,IsPPlateApproved,SaleCategory,Branch,FamilyCode,FuelTypeDescription,Cylinders,FuelDeliveryDescription,EngineTypeDescription,InductionDescription,OptionCategory,YearGroup,Make,LimitedEdition,ImportFlag,CurrentRelease,Description,MonthGroup,SequenceNum,EngineSize,EngineDescription,Model,BodyStyleDescription,DriveCode,DriveDescription,GearTypeDescription,DoorNum,MakeCode,dataset_split,AgeDays
0,100000.0,1690.0,3.0,V60,HBA04 ######,VE,225/60 R16,225/60 R16,73.0,16x7.0,16x7.0,1476.0,4894.0,1899.0,6G1EK52B#8L######,4.0,Large,Passenger,2600.0,330.0,DOHC with VVT,2915.0,6000.0,180.0,White,93.0,5.0,Electronic Sequential,140.0,230.0,Front,4.0,227878.0,Floor,2000.0,34790.0,AUSTRALIA,4 Stroke,T,Auction,Perth (WA),COMMODO,Petrol,6,Multi-Point Injection,Piston,Aspirated,PASS,2008,Holden,F,L,F,VE Omega Sedan 4dr. Auto 4sp 3.6i,0,0,3565,3.6,Commodore,Sedan,RWD,Rear Wheel Drive,Automatic,4,HOLD,train,2832.0
1,100000.0,1380.0,3.0,V90,VH-######,VR,205/65 R15,205/65 R15,68.0,15x6.0,15x6.0,1478.0,4903.0,1794.0,6H8VRK35HPL######,2.0,Large,Passenger,3200.0,295.0,Pushrod,2822.0,4800.0,130.0,Red,86.0,5.0,Electronic,360.0,600.0,Front,4.0,153091.0,Floor,6800.0,27978.0,AUSTRALIA,4 Stroke,T,Auction,Belmore (NSW),COMMODO,Petrol,6,Multi-Point Injection,Piston,Aspirated,PASS,1993,Holden,F,L,F,VR Executive Wagon 5dr. Auto 4sp 3.8i,7,41,3791,3.8,Commodore,Wagon,RWD,Rear Wheel Drive,Automatic,5,HOLD,train,2635.0
2,100000.0,1545.0,3.0,In-line,2AZ-#######,ACA33R,225/65 R17,225/65 R17,60.0,17x6.5,17x6.5,1695.0,4625.0,1815.0,JTMBD33V*05######,4.0,Medium,SUV,4000.0,224.0,DOHC with VVT,2660.0,6000.0,125.0,040 - Glacier White (T),16.0,5.0,Electronic Sequential,80.0,130.0,Front,5.0,27374.0,Floor,22900.0,31990.0,JAPAN,4 Stroke,T,Dealer Only Auction,Sunshine (VIC),RAV4,Petrol,4,Multi-Point Injection,Piston,Aspirated,SUV,2012,Toyota,F,L,F,ACA33R MY12 CV Wagon 5dr Man 5sp 4x4 2.4i,0,6,2362,2.4,RAV4,Wagon,4XO,4X4 On Demand,Manual,5,TOYO,train,492.0


KerbWeight                        2581
EngineConfigurationDescription    1829
Series                            1001
EngineNum                          967
FrontTyreSize                      869
RearTyreSize                       869
Height                             706
RearRimDesc                        695
FrontRimDesc                       695
Length                             651
Width                              641
VIN                                583
ValvesCylinder                     490
VFactsClass                        391
VFactsSegment                      391
dtype: int64

In [8]:
model_df.to_csv(MODEL_PATH, index=False)
print('Saved model dataframe:', MODEL_PATH)


Saved model dataframe: /home/localhost/datium_model/data/processed/dataset_model.csv


## 3) Training matrix (label encoding + log transforms)


In [9]:
training_df, label_maps = to_training_matrix(
    model_df,
    label_map_path=LABEL_MAP_PATH,
    log_target=True,
    log_newprice=True,
)
print('Training matrix shape:', training_df.shape)
print('Label map columns saved:', len(label_maps))
display(training_df.head(3))



Training matrix shape: (57800, 67)
Label map columns saved: 39


,WarrantyKM,KerbWeight,WarrantyYears,EngineConfigurationDescription,EngineNum,Series,FrontTyreSize,RearTyreSize,FuelCapacity,FrontRimDesc,RearRimDesc,Height,Length,Width,VIN,ValvesCylinder,VFactsSegment,VFactsClass,TorqueRPMTo,Torque,CamDescription,WheelBase,PowerRPMTo,Power,Colour,Age_Comp_Months,SeatCapacity,MethodOfDeliveryDescription,GoodKM,AverageKM,EngineLocation,GearNum,KM,GearLocationDescription,Sold_Amount,NewPrice,BuildCountryOriginDescription,EngineCycleDescription,IsPPlateApproved,SaleCategory,Branch,FamilyCode,FuelTypeDescription,Cylinders,FuelDeliveryDescription,EngineTypeDescription,InductionDescription,OptionCategory,YearGroup,Make,LimitedEdition,ImportFlag,CurrentRelease,Description,MonthGroup,SequenceNum,EngineSize,EngineDescription,Model,BodyStyleDescription,DriveCode,DriveDescription,GearTypeDescription,DoorNum,MakeCode,dataset_split,AgeDays
0,100000.0,1690.0,3.0,4,1241,978,176,177,73.0,30,33,1476.0,4894.0,1899.0,338,4.0,1,2,2600.0,330.0,0,2915.0,6000.0,180.0,1071,93.0,5.0,1,140.0,230.0,0,4.0,227878.0,2,7.601402,10.457114,1,1,1,0,12,191,2,6,5,0,0,2,2008,17,0,0,0,8539,0,0,3565,29,197,10,6,6,0,4,17,1,2832.0
1,100000.0,1380.0,3.0,8,1621,1002,113,115,68.0,15,18,1478.0,4903.0,1794.0,623,2.0,1,2,3200.0,295.0,8,2822.0,4800.0,130.0,865,86.0,5.0,0,360.0,600.0,0,4.0,153091.0,2,8.824825,10.239210,1,1,1,0,4,191,2,6,5,0,0,2,1993,17,0,0,0,8870,7,41,3791,31,197,16,6,6,0,5,17,1,2635.0
2,100000.0,1545.0,3.0,1,229,91,181,182,60.0,37,40,1695.0,4625.0,1815.0,2376,4.0,4,3,4000.0,224.0,0,2660.0,6000.0,125.0,1,16.0,5.0,1,80.0,130.0,0,5.0,27374.0,2,10.038936,10.373210,15,1,1,1,15,486,2,4,5,0,0,3,2012,49,0,0,0,766,0,6,2362,17,469,16,4,3,2,5,50,1,492.0


In [10]:
training_df.to_csv(TRAINING_MATRIX_PATH, index=False)
print('Saved training matrix:', TRAINING_MATRIX_PATH)


Saved training matrix: /home/localhost/datium_model/data/processed/dataset_training_matrix.csv


In [11]:
training_df['dataset_split']

0        1
1        1
2        1
3        1
4        1
        ..
57795    0
57796    0
57797    0
57798    0
57799    0
Name: dataset_split, Length: 57800, dtype: int64

In [12]:
train,test = training_df[training_df['dataset_split']==1], training_df[training_df['dataset_split']==0]
X_train,y_train = train.drop(columns=['Sold_Amount','dataset_split']), train['Sold_Amount']
X_test,y_test = test.drop(columns=['Sold_Amount','dataset_split']), test['Sold_Amount']


In [13]:
len(y_train)
len(y_test)

10292

In [14]:
# Feature Selection 
# Method 1: SelectKBest 
from sklearn.feature_selection import SelectKBest, f_regression
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

top_feature_num = 20
selector = SelectKBest(score_func=f_regression, k=top_feature_num)
fit = selector.fit(X_train, y_train)

# Get the transformed data
X_new = fit.transform(X_train)
# print("Selected features shape:", X_new.shape)
# print("Scores of the features:", fit.scores_)

filter = selector.get_support()
TOP_FEATURES = list(X_train.columns[filter])
print(f"Selected features: {TOP_FEATURES}")


model = lgbm.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
model.fit(X_train[TOP_FEATURES], y_train)
print(f"New Train R2: {model.score(X_train[TOP_FEATURES], y_train):.4f}")
train_rmse = float(np.sqrt(mean_squared_error(y_train, model.predict(X_train[TOP_FEATURES]))))
print(f"New Train RMSE: {train_rmse:.4f}")


print(f"New Test  R2: {model.score(X_test[TOP_FEATURES], y_test):.4f}")
test_rmse = float(np.sqrt(mean_squared_error(y_test, model.predict(X_test[TOP_FEATURES]))))
print(f"New Test RMSE: {test_rmse:.4f}")

Selected features: ['KerbWeight', 'FrontTyreSize', 'RearTyreSize', 'FuelCapacity', 'FrontRimDesc', 'RearRimDesc', 'Height', 'Width', 'Torque', 'Age_Comp_Months', 'GoodKM', 'AverageKM', 'KM', 'NewPrice', 'FuelTypeDescription', 'InductionDescription', 'YearGroup', 'DriveCode', 'DriveDescription', 'AgeDays']
New Train R2: 0.9164
New Train RMSE: 0.1834
New Test  R2: 0.8798
New Test RMSE: 0.2206


In [15]:
# Method 1: SHAP
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error
model = lgbm.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
model.fit(X_train, y_train)
print(f"Train R2: {model.score(X_train, y_train):.4f}")
train_rmse = float(np.sqrt(mean_squared_error(y_train, model.predict(X_train))))
print(f"Train RMSE: {train_rmse:.4f}")


print(f"Test  R2: {model.score(X_test, y_test):.4f}")
test_rmse = float(np.sqrt(mean_squared_error(y_test, model.predict(X_test))))
print(f"Test RMSE: {test_rmse:.4f}")

Train R2: 0.9297
Train RMSE: 0.1682
Test  R2: 0.8969
Test RMSE: 0.2044


In [16]:
import shap
explainer = shap.Explainer(model)
shap_values = explainer(X_test)
shap_importance = shap_values.abs.mean(0).values
importance_df = pd.DataFrame({'features': X_train.columns,
                              'importance': shap_importance})
importance_df.sort_values(by='importance', ascending=False, inplace=True)
importance_df.head(n=20)
# top 20 features have SHAP importance > 0.008

/home/localhost/datium_model/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,features,importance
34,NewPrice,0.221409
64,AgeDays,0.185235
32,KM,0.135378
25,Age_Comp_Months,0.091472
11,Height,0.068139
63,MakeCode,0.025659
47,YearGroup,0.024016
48,Make,0.022726
35,BuildCountryOriginDescription,0.020534
29,AverageKM,0.016189


In [17]:
# Lets try with these 20 features from SHAP
TOP_FEATURES = list(importance_df.features)[:20]

import lightgbm as lgbm
from sklearn.metrics import mean_squared_error
model = lgbm.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
model.fit(X_train[TOP_FEATURES], y_train)
print(f"New Train R2: {model.score(X_train[TOP_FEATURES], y_train):.4f}")
train_rmse = float(np.sqrt(mean_squared_error(y_train, model.predict(X_train[TOP_FEATURES]))))
print(f"New Train RMSE: {train_rmse:.4f}")


print(f"New Test  R2: {model.score(X_test[TOP_FEATURES], y_test):.4f}")
test_rmse = float(np.sqrt(mean_squared_error(y_test, model.predict(X_test[TOP_FEATURES]))))
print(f"New Test RMSE: {test_rmse:.4f}")


New Train R2: 0.9256
New Train RMSE: 0.1730
New Test  R2: 0.8932
New Test RMSE: 0.2080


In [18]:
list(importance_df.features)[:20]

['NewPrice',
 'AgeDays',
 'KM',
 'Age_Comp_Months',
 'Height',
 'MakeCode',
 'YearGroup',
 'Make',
 'BuildCountryOriginDescription',
 'AverageKM',
 'KerbWeight',
 'SaleCategory',
 'Branch',
 'RearRimDesc',
 'VIN',
 'PowerRPMTo',
 'FrontRimDesc',
 'GoodKM',
 'RearTyreSize',
 'VFactsSegment']

Overall the 20 features chosen: SHAP is better in terms of outcomes

['NewPrice',
 'AgeDays',
 'KM',
 'Age_Comp_Months',
 'Height',
 'MakeCode',
 'YearGroup',
 'Make',
 'BuildCountryOriginDescription',
 'AverageKM',
 'KerbWeight',
 'SaleCategory',
 'Branch',
 'RearRimDesc',
 'VIN',
 'PowerRPMTo',
 'FrontRimDesc',
 'GoodKM',
 'RearTyreSize',
 'VFactsSegment']